In [158]:
import tabula
import pandas as pd

In [218]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import os
from bs4 import BeautifulSoup
from random import uniform

In [160]:
%matplotlib notebook
import matplotlib.pyplot as plt

### Welcher Kanton hat am meisten Bäckereien?

1. Wir definieren eine Variabel, bspw. "Garage", "Bäckerei", etc.
2. Wir suchen bei Zefix.ch danach.
3. Wir laden das PDF herunter und speichern es ab.
4. Wir ziehen alle Elemtene aus der Liste.
5. Wir reinigen die Daten.
6. Bäckerein nach Kanton
7. Wir vereinigen (mergen) die Daten mit Bevölkerungsdaten nach Kanton, allenfalls "nur" die deutschsprachigen Kantone.

1. Wir definieren eine Variabel

In [369]:
elem = "Bäckerei"

2. Wir suchen Zefix danach ab.

In [174]:
#Wir starten den Browser auf
driver = webdriver.Chrome(options=chromeOptions)

In [175]:
#Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
driver.get('https://www.zefix.ch')

In [177]:
#Wir benutzen den Webinspector
search = driver.find_element_by_id('firm-name-fomfield')

In [178]:
#Send search
search.send_keys(elem)

In [179]:
#Submit Search Button
driver.find_element_by_id('submit-search-btn').click()

In [180]:
#Wait
time.sleep(1.5)

3. Wir laden das PDF herunter und speichern es ab

In [181]:
#Press print button
print_button = driver.find_element_by_class_name('print-button')

In [182]:
print_button.click()

In [ ]:
#Nun von Hand herunterladen

4. Ziehen wir alle Elemente aus der Liste

In [345]:
tables1 = tabula.read_pdf('20201102-bäckerei.pdf', stream=True, pages = 1)
headers = ['Zahl', 'Bäckerei', 'Firmenart', 'Sitz', 'CHE', 'UID']
tables1[0].columns = headers
tables1[0].head()

,Zahl,Bäckerei,Firmenart,Sitz,CHE,UID
0,2,Bäckerei Abderhalden AG Wattwil,Ltd,Wattwil,SG,CHE-107.254.772
1,3,Bäckerei Aebersold GmbH,LLC,Murten,FR,CHE-100.206.039
2,4,Bäckerei A. Gubser,SP,Zollikon,ZH,CHE-337.549.457
3,5,Bäckerei Allenspach GmbH,LLC,Fischingen,TG,CHE-265.077.575
4,6,Bäckerei alpascha GmbH,LLC,Münchenstein,BL,CHE-198.680.660


In [347]:
tables2 = tabula.read_pdf('20201102-bäckerei.pdf', stream=True, pages = 2)
del tables2[0]['Unnamed: 0']
headers = ['Zahl', 'Bäckerei', 'Firmenart/Sitz', 'CHE/UID']
tables2[0].columns = headers
tables2[0]['CHE'] = tables2[0]['CHE/UID'].str.split(expand=True)[0]
tables2[0]['UID'] = tables2[0]['CHE/UID'].str.split(expand=True)[1]
del tables2[0]['CHE/UID']
tables2[0]['Firmenart'] = tables2[0]['Firmenart/Sitz'].str.split(expand=True)[0]
tables2[0]['Sitz'] =  tables2[0]['Firmenart/Sitz'].str.split(expand=True)[1] + tables2[0]['Firmenart/Sitz'].str.split(expand=True)[2].fillna(value="")
del tables2[0]['Firmenart/Sitz']
tables2[0].head()

,Zahl,Bäckerei,CHE,UID,Firmenart,Sitz
0,NaN,MOSER,NaN,NaN,NaN,NaN
1,42.0,Bäckerei Confiserie Bucher AG,LU,CHE-115.564.068,Ltd,Emmen
2,43.0,Bäckerei-Confiserie-Café Brioche KLG,BE,CHE-243.694.340,GP,Kirchberg(BE)
3,44.0,Bäckerei-Confiserie-Café Kuhnen GmbH,BE,CHE-107.208.476,LLC,Lenk
4,45.0,Bäckerei Confiserie Galli AG,BE,CHE-114.972.619,Ltd,Steffisburg


In [352]:
tables3 = tabula.read_pdf('20201102-bäckerei.pdf', stream=True, pages = 3)
del tables3[0]['Unnamed: 0']
headers = ['Bäckerei', 'Firmenart', 'Sitz', 'CHE', 'UID']
tables3[0].columns = headers
tables3[0]['Zahl'] = tables3[0]['Bäckerei'].str.split(expand=True)[0]
tables3[0].head()

,Bäckerei,Firmenart,Sitz,CHE,UID,Zahl
0,96 Bäckerei-Konditorei Alt,SP,Endingen,AG,CHE-105.776.284,96
1,97 Bäckerei-Konditorei A. Zuber AG,Ltd,Stalden (VS),VS,CHE-332.997.417,97
2,98 Bäckerei Konditorei Bachmann GmbH,LLC,Rheineck,SG,CHE-391.154.650,98
3,"99 Bäckerei-Konditorei Beck Beck, Inh. H. Gmür",SP,St. Gallen,SG,CHE-106.057.387,99
4,100 Bäckerei-Konditorei Bieri Alfred,SP,Erlenbach im Simmental,BE,CHE-105.986.692,100


In [354]:
tables4 = tabula.read_pdf('20201102-bäckerei.pdf', stream=True, pages = 4)
headers = ['Bäckerei', 'Firmenart', 'Sitz', 'CHE', 'UID']
tables4[0].columns = headers
tables4[0]['Zahl'] = tables4[0]['Bäckerei'].str.split(expand=True)[0]
tables4[0].head()
tables4[0].head()

,Bäckerei,Firmenart,Sitz,CHE,UID,Zahl
0,"151 Bäckerei, Konditorei Fahrni",SP,Dotzigen,BE,CHE-107.180.240,151
1,"152 Bäckerei Konditorei Fahrni, Nachfolger Hei...",SP,Thun,BE,CHE-276.039.461,152
2,153 BÄCKEREI - KONDITOREI FÄSSLER,SP,Rüte,AI,CHE-115.132.898,153
3,154 Bäckerei-Konditorei Forrer,SP,Ebnat-Kappel,SG,CHE-107.972.666,154
4,155 Bäckerei-Konditorei Frei AG,Ltd,Obersiggenthal,AG,CHE-109.472.587,155


In [355]:
tables5 = tabula.read_pdf('20201102-bäckerei.pdf', stream=True, pages = 5)
headers = ['Bäckerei', 'Firmenart', 'Sitz', 'CHE', 'UID']
tables5[0].columns = headers
tables5[0]['Zahl'] = tables5[0]['Bäckerei'].str.split(expand=True)[0]
tables5[0].head()
tables5[0].head()

,Bäckerei,Firmenart,Sitz,CHE,UID,Zahl
0,206 Bäckerei-Konditorei Markus Bartlome,SP,Hindelbank,BE,CHE-107.182.262,206
1,207 Bäckerei-Konditorei Markus Zimmermann,SP,Obergösgen,SO,CHE-106.933.439,207
2,208 Bäckerei-Konditorei Merz AG,Ltd,Luzern,LU,CHE-106.505.983,208
3,209 Bäckerei-Konditorei Meyer KlG,GP,Brügg,BE,CHE-453.139.911,209
4,210 Bäckerei-Konditorei Mischler,SP,Selzach,SO,CHE-109.834.874,210


In [357]:
tables6 = tabula.read_pdf('20201102-bäckerei.pdf', stream=True, pages = 6)
tables6[0].columns = headers
tables6[0]['Zahl'] = tables6[0]['Bäckerei'].str.split(expand=True)[0]
tables6[0].head()
tables6[0].head()

,Bäckerei,Firmenart,Sitz,CHE,UID,Zahl
0,260 Bäckerei - Konditorei Unternährer & Co,GP,Kappel (SO),SO,CHE-112.494.017,260
1,261 Bäckerei-Konditorei U. Schnidrig,SP,Kandergrund,BE,CHE-107.201.014,261
2,262 Bäckerei-Konditorei Vetterli,SP,Horgen,ZH,CHE-285.280.174,262
3,263 Bäckerei-Konditorei Vincenz AG,Ltd,Lauterbrunnen,BE,CHE-402.020.872,263
4,264 Bäckerei-Konditorei von Burg AG,Ltd,Küsnacht (ZH),ZH,CHE-396.408.680,264


In [358]:
tables7 = tabula.read_pdf('20201102-bäckerei.pdf', stream=True, pages = 7)
tables7[0].columns = headers
tables7[0]['Zahl'] = tables7[0]['Bäckerei'].str.split(expand=True)[0]
tables7[0].head()
tables7[0].head()

,Bäckerei,Firmenart,Sitz,CHE,UID,Zahl
0,315 Bäckerei Schwegler GmbH,LLC,Willisau,LU,CHE-107.804.537,315
1,316 Bäckerei Schwyter AG,Ltd,St. Gallen,SG,CHE-109.088.421,316
2,317 Bäckereiservice Hunkeler,SP,Niederwil (AG),AG,CHE-110.602.224,317
3,318 Bäckerei Siegfried,SP,Zäziwil,BE,CHE-103.415.409,318
4,319 Bäckerei Speck,SP,Oberkulm,AG,CHE-113.973.800,319


In [366]:
frames = [tables1[0], tables2[0], tables3[0], tables4[0], tables5[0], tables6[0], tables7[0]]
df = pd.concat(frames)
df = df.reset_index()
del df['Zahl']
del df['index']

6. Bäckereien nach Kanton

In [368]:
df['CHE'].value_counts()

BE    84
SG    39
AG    39
ZH    35
LU    30
GR    15
SO    14
TG    12
FR    12
VS    11
BS     9
SZ     8
BL     7
AR     6
ZG     4
UR     3
GL     2
SH     2
AI     1
OW     1
Name: CHE, dtype: int64

In [373]:
df['Sitz'].value_counts().head(10)

Zürich         8
Basel          8
Luzern         5
Bern           5
Thun           5
Köniz          5
St. Gallen     4
Schangnau      3
Winterthur     3
Niederglatt    3
Name: Sitz, dtype: int64

7. Nach Bevölkerung

In [379]:
df_bev = pd.read_csv('px-x-0103030000_102.csv', encoding = 'latin', delimiter=';', header=1)

In [ ]:
['He']

In [384]:
def splitonfirst(elem):
    elem = elem.split(' ', 1)[1]
    return elem

df_bev['Gemeinde'] = df_bev['Kanton (-) / Bezirk (>>) / Gemeinde (......)'].apply(splitonfirst)

In [386]:
df_bev = df_bev[['Gemeinde', 'Alter - Total']].copy()

In [388]:
df_bev.head().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Gemeinde       5 non-null      object
 1   Alter - Total  5 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


In [391]:
beck_count = pd.DataFrame(df['Sitz'].value_counts())
beck_count = beck_count.reset_index()

In [406]:
dfFinal = beck_count.merge(df_bev, left_on='index', right_on='Gemeinde')
del dfFinal['index']
dfFinal['Beck pro 100000'] = dfFinal['Sitz'] / dfFinal['Alter - Total'] * 1000
dfFinal.sort_values(by='Beck pro 100000', ascending = False)

,Sitz,Gemeinde,Alter - Total,Beck pro 100000
214,1,Simplon,300,3.333333
7,3,Schangnau,905,3.314917
157,1,Kippel,318,3.144654
50,1,Ulmiz,449,2.227171
24,2,Tujetsch,1207,1.657001
...,...,...,...,...
60,1,Uster,35007,0.028566
65,1,Chur,35470,0.028193
8,3,Winterthur,113173,0.026508
105,1,Fribourg,38197,0.026180
